In [1]:
import pandas as pd
import numpy as np

Reading in the training and test data we well as the column name to column label mapping file created in `column_name.ipynb`

In [350]:
codebook=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\column_name_file_clean.csv")
train=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\Data\train_clean.csv")
test=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\Data\test_clean.csv",index_col=False)
cc=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\country_continent.csv",index_col=False)

C:\Users\Jack\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,4,6,10,12,18,24,35,56,57,65,85,98,99,100,105,124,125,126,127,128,130,131,132,133,166,171,172,228,229,230,240,256,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Jack\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,6,10,12,21,24,35,85,100,124,126,127,128,130,131,133,166,172,177,220,222,228,229,230,240,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [351]:
train=train.merge(cc,how='left',right_on='iso 3166 country',left_on='v17')
train=train.drop(['iso 3166 country','v17','v78','v161','v20','v154','v155','v25'],axis=1)
test=test.merge(cc,how='left',right_on='iso 3166 country',left_on='v17')
test=test.drop(['iso 3166 country','v17','v78','v161','v20','v154','v155','v25'],axis=1)

In [352]:
def kids_adult_kids(x):
    x['g_2']=x['v90']+x['v208']
    x['g_3']=x['v91']+x['v209']
    x['g_4']=x['v92']+x['v210']
    x['g_5']=x['v93']+x['v211']
    x['g_6']=x['v94']+x['v212']
    x['g_7']=x['v95']+x['v213']
    x['g_8']=x['v96']+x['v214']
    x['g_9']=x['v97']+x['v215']
    x['g_10']=x['v86']+x['v204']
    x['g_11']=x['v87']+x['v205']
    x['g_12']=x['v88']+x['v206']
    x['g_13']=x['v89']+x['v207']
    
    for combo in ['11','21','12','22','13','23','14','24','15','25','16','26']:
        x['total'+combo]=1*pd.DataFrame(x[['g_2','g_3','g_4','g_5','g_6','g_7','g_8','g_9','g_10',
                                           'g_11','g_12','g_13']]==combo).sum(axis=1)
        
    rel=train[['v204','v205','v206','v207','v208','v209','v210','v211','v212','v213','v214','v215']].copy()
    age=train[['v259','v260','v261','v262','v263','v264','v265','v266','v267','v268','v269','v270']].copy()

    rel.columns=[10,11,12,13,2,3,4,5,6,7,8,9]
    age.columns=[10,11,12,13,2,3,4,5,6,7,8,9]

    age[age.isna()]=0
    age[age.isin(['Not available','Refusal',"Don't know",'No answer','No Answer','Not applicable'])]=0

    rel[rel.isna()]=0
    rel[age.isin(['Not available','Refusal',"Don't know",'No answer',"No Answer",'Not applicable'])]=0

    for c in [10,11,12,13,2,3,4,5,6,7,8,9]:
        try:
            age[c]=age[c].astype(float)
        except ValueError:
            print(c+" is not numeric")
        
    x['adult_children']=((1*(age<=1990))*(1*(rel=="2"))).sum(axis=1)
    x['kids']=((1*(age>2000))).sum(axis=1)
    
   
    x=x.drop(['g_2','g_3','g_4','g_5','g_6','g_7','g_8','g_9','g_10','g_11','g_12','g_13'],axis=1)
    
    return(x)

In [353]:
train=kids_adult_kids(train)
test=kids_adult_kids(test)

In [354]:
for c in train.columns.drop('satisfied'):
    try:
        train.loc[train[c].isin(['Not applicable']),c]=-999
        
        ### Addition
        test.loc[test[c].isin(['Not applicable']),c]=-999
        ####
    except ValueError:
        print(c+" is not numeric")

For all columns, values of `.a,.b,.c,.d` or  `.` have different meanings, typically `Nothing, Not applicable, Refusal, Don't know` or `No answer`. 

For we create an indicator variable for each column specifying whether each respondant responded with any of these values. For the original column, we replace this `.a,.b,.c,.d` or  `.` value with `na`.

For Example: Respondant 9948 has responce `.a` for `v263: Year of birth of second person in household`. So we create column `v263given`  9948 will have value 1 `v263` 9948 will now have value `NaN`. Respondents who did not have `.a,.b,.c,.d` or  `.` provided for `v263` will have 0 in `v263given` and the original value in `v263`.

In [355]:
for c in train.columns.drop('satisfied'):
    train[c+'given']=train[c].isin(['Not available','Refusal',"No Answer","Don't know",'No answer'])*1
    train.loc[train[c].isin(['Not available','Refusal',"No Answer","Don't know",'No answer']),c]=np.nan
    test[c+'given']=test[c].isin(['Not available','Refusal',"No Answer","Don't know",'No answer'])*2
    test.loc[test[c].isin(['Not available','Refusal',"No Answer","Don't know",'No answer']),c]=np.nan

Values are sometimes recorded in two datatypes. for example we observe the unique values for `v10: Born in country`:

In [356]:
train['v10'].unique()

array([1, 2, '1', '2', nan], dtype=object)

We see 1 and '1' are both possible values, to correct for this we decide to change the string values to be numerics:

In [357]:
for c in train.columns.drop('satisfied'):
    try:
        train[c]=train[c].astype(float)
        
        ### Addition
        test[c]=test[c].astype(float)
        ####
    except ValueError:
        print(c+" is not numeric")

cntry is not numeric
InterviewDayOfWeek is not numeric
continent code is not numeric


We can now check again to make sure this issue has been corrected:

In [358]:
train['v10'].unique()

array([ 1.,  2., nan])

All variables which are not `float` do not need to be imputed as NA can be left as a value which will be converted into a dummy variable.

In [359]:
train.columns[train.dtypes!='float64']

Index(['cntry', 'satisfied', 'InterviewDayOfWeek', 'continent code'], dtype='object')

Make a separate training set with only numeric values

In [360]:
num_col=train.columns[~train.columns.isin(train.columns[train.dtypes!='float64'])]
train_n=train[num_col]
train_n=train_n.drop(columns={'id'})

#### Addition
num_col=test.columns[~test.columns.isin(test.columns[test.dtypes!='float64'])]
test_n=test[num_col]
test_n=test_n.drop(columns={'id'})

In [361]:
train_n.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,total13given,total23given,total14given,total24given,total15given,total25given,total16given,total26given,adult_childrengiven,kidsgiven
0,2.0,2.0,74.0,11010.0,-999.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,2.0,58.0,11010.0,-999.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6.0,2.0,47.0,11010.0,11010.0,2.0,2.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.0,2.0,22.0,11010.0,-999.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,24.0,11010.0,-999.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Using `Imputer` from `sklearn` we impute these numerical columns with the mean.

In [362]:
test_n.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,total13given,total23given,total14given,total24given,total15given,total25given,total16given,total26given,adult_childrengiven,kidsgiven
0,0.0,1.0,63.0,15020.0,-999.0,2.0,2.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,2.0,44.0,21070.0,21070.0,1.0,2.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.0,2.0,21.0,11010.0,11010.0,2.0,1.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,2.0,75.0,11010.0,-999.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,22.0,11010.0,11010.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [363]:
test_n.nunique()[test_n.nunique()==0]

Series([], dtype: int64)

We create our response for our training data, and remove this variable from our training data

In [364]:
y_train=train['satisfied']

# Imputation

In [365]:
for c in train_n.columns:
    try:
        train_n.loc[train_n[c].isna(),c]=train_n.loc[train_n[c].isin([-999,np.nan]),c].mean()
        
        ### Addition
        test_n.loc[test_n[c].isna(),c]=test_n.loc[test_n[c].isin([-999,np.nan]),c].mean()
        ####
    except ValueError:
        print(c+" is not numeric")
        
## Addition
for v in ['v89','v195','v207','v262']:
    test_n[v]=(train_n[v]).mean()

Adding -999 to na

In [366]:
for c in train_n.columns:
    try:
        train_n.loc[train_n[c].isin([-999]),c]=np.nan
        
        ### Addition
        test_n.loc[test_n[c].isin([-999]),c]=np.nan
        ####
    except ValueError:
        print(c+" is not numeric")

Make a separate training set with only the character variables:

In [367]:
train_c=train[train.columns[(train.dtypes!='float64')&(train.columns!='satisfied')]]

## Addition
test_c=test[test.columns[(test.dtypes!='float64')&(test.columns!='satisfied')]]

For these character variables, we convert them to dummy variables where each level will have it's own variable with levels:
- 0 if the respondent did not give that level 
- 1 if the respondent gave that level 

In [368]:
train_c=pd.get_dummies(train_c)

## Addition
test_c=pd.get_dummies(test_c)

We now append all these dummy variable columns to our `train_n` dataframe to create a finalized training set, with no missing data in it. 

In [369]:
for v in train_c.columns[~train_c.columns.isin(test_c.columns)]:
    print(v)

In [370]:
for c in train_c.columns:
    train_n[c]=train_c[c]
    
    ## Addition
    test_n[c]=test_c[c]

## Addition
for v in train_c.columns[~train_c.columns.isin(test_c.columns)]:
    test_n[v]=0


In [410]:
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

In [411]:
gbm = GradientBoostingClassifier(
    n_estimators= 5,
    max_depth= 3,
    learning_rate= 0.02,
    loss='deviance')

In [412]:
gbm.fit(train_n,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [576]:
imp=pd.DataFrame({'name':train_n.columns,'imp':gbm.feature_importances_})

In [551]:
pred=pd.DataFrame(gbm.predict_proba(test_n))

In [552]:
pred['id']=test['id'].astype(int)

In [553]:
pd.DataFrame({'id':pred['id'],'Predicted':pred[1]}).to_csv(r"C:\Users\Jack\Desktop\Project441\pred1.csv",index = False)

# XGBoost

In [374]:
import xgboost as xgb
param = {'max_depth': 3, 'eta': 0.02, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
num_round = 500

In [375]:
dtrain = xgb.DMatrix(train_n, label=y_train, missing=np.nan)

In [ ]:
xg = xgb.XGBClassifier()
fit = xg.fit(train_n, y_train)
fit.feature_importances_

In [376]:
bst = xgb.train(param, dtrain, 500)

In [400]:
train_n.shape

(30080, 599)

In [401]:
test_n.shape

(9245, 599)

In [379]:
dtest = xgb.DMatrix(test_n)
ypred = bst.predict(dtest)

In [419]:
pd.DataFrame({'id':test['id'].astype(int),'Predicted':ypred}).to_csv(r"C:\Users\Jack\Desktop\Project441\pred_xgb.csv",index = False)

In [60]:
train_n.to_csv(r"C:\Users\Jack\Desktop\Project441\xgb_training.csv",index = False)

In [94]:
test_n.to_csv(r"C:\Users\Jack\Desktop\Project441\xgb_test.csv",index = False)

In [381]:
imp=pd.DataFrame.from_dict(bst.get_score(importance_type='gain'), orient='index')
imp=imp.reset_index()
imp.columns=['feature','importance']

In [86]:
train_n.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,cntry_PT,cntry_SE,cntry_SI,InterviewDayOfWeek_Friday,InterviewDayOfWeek_Monday,InterviewDayOfWeek_Saturday,InterviewDayOfWeek_Sunday,InterviewDayOfWeek_Thursday,InterviewDayOfWeek_Tuesday,InterviewDayOfWeek_Wednesday
0,2.0,2.0,74.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0,0,0,0,0,0,0,0,0,1
1,4.0,2.0,58.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0,0,0,0,0,1,0,0,0,0
2,6.0,2.0,47.0,11010.0,11010.0,2.0,2.0,1.0,2.0,1.0,...,0,0,0,0,0,0,0,0,0,1
3,10.0,2.0,22.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,1.0,24.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
pd.DataFrame.from_dict(data)

In [829]:
imp=pd.DataFrame({'name':train_n.columns,'imp':model.feature_importances_})
imp.sort_values("imp", ascending=False)

XGBoostError: need to call fit or load_model beforehand

In [830]:
train_n.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,cntry_PT,cntry_SE,cntry_SI,InterviewDayOfWeek_Friday,InterviewDayOfWeek_Monday,InterviewDayOfWeek_Saturday,InterviewDayOfWeek_Sunday,InterviewDayOfWeek_Thursday,InterviewDayOfWeek_Tuesday,InterviewDayOfWeek_Wednesday
0,2.0,2.0,74.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0,0,0,0,0,0,0,0,0,1
1,4.0,2.0,58.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0,0,0,0,0,1,0,0,0,0
2,6.0,2.0,47.0,11010.0,11010.0,2.0,2.0,1.0,2.0,1.0,...,0,0,0,0,0,0,0,0,0,1
3,10.0,2.0,22.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,1.0,24.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
pd.DataFrame({'id':pred['id'],'Predicted':pred[1]}).to_csv(r"C:\Users\Jack\Desktop\Project441\pred1.csv",index = False)

In [7]:
train['num_in_house']=train.iloc[:,258:271].max(axis=1)

In [8]:
train.columns[258:271]

Index(['v258', 'v259', 'v260', 'v261', 'v262', 'v263', 'v264', 'v265', 'v266',
       'v267', 'v268', 'v269', 'v270'],
      dtype='object')

In [9]:
train.columns[258:271][6]

'v264'

In [10]:
train['num_in_house2']=0
for c in range(0, 13):
    train['num_in_house2']=train['num_in_house2']+(train[train.columns[258:271][c]]==train['num_in_house'])*[1,10,11,12,13,2,3,4,5,6,7,8,9][c]

In [12]:
v='cntry'

g=pd.DataFrame(train.groupby([v])['satisfied'].mean())
g=g.reset_index()
g2=pd.DataFrame(train.groupby([v])['satisfied'].count())
g2=g2.rename(columns={'satisfied':'n'})
g2=g2.reset_index()
g=g.merge(g2)
g.sort_values('satisfied',ascending=False)

,cntry,satisfied,n
5,DK,0.816429,1193
8,FI,0.732002,1653
2,CH,0.729184,1189
16,NO,0.696113,1132
19,SE,0.692469,1434
15,NL,0.627164,1502
4,DE,0.604690,2388
13,IL,0.580437,2014
0,AT,0.570296,1387
1,BE,0.555556,9


In [13]:
g2

,cntry,n
0,AT,1387
1,BE,9
2,CH,1189
3,CZ,1683
4,DE,2388
5,DK,1193
6,EE,1606
7,ES,1520
8,FI,1653
9,FR,1502


In [262]:
import sklearn
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import numpy as np

In [263]:
target = 'satisfied'
IDcol = 'id'

In [264]:
gbm = GradientBoostingClassifier(
    n_estimators= 500,
    max_depth= 3,
    learning_rate= 0.02,
    loss='deviance')


In [184]:
from sklearn.preprocessing import Imputer
imputer = Imputer()
imputer.fit_transform(train)

In [30]:
gbm.feature_importances_

array([0.14177209, 0.13667957, 0.72154834])

In [ ]:
gbm.score(X_val, y_val)

In [37]:
train['v10'].unique()

array(['1', '2', 'nan'], dtype=object)

In [58]:
a=gbm.predict(pd.get_dummies(train['v10','v79']))

In [59]:
pd.Series(a).value_counts()

1    30073
0        7
dtype: int64

In [62]:
train['satisfied'].value_counts()

1    15626
0    14454
Name: satisfied, dtype: int64